<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/BLOCKCHAIN_AAI_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import hashlib
import json
import time
import os
import re # Import for regular expressions to parse LLM response
# Import the necessary libraries for Gemini API
from google.colab import userdata
import google.generativeai as genai

# --- Configure Gemini API ---
# IMPORTANT: Replace 'GEMINI' with the actual secret name if different in your Colab environment.
# Or, if running outside Colab, set this as an environment variable.
try:
    GOOGLE_API_KEY = userdata.get('GEMINI')
    genai.configure(api_key=GOOGLE_API_KEY)
    print("Gemini API configured successfully.")
except Exception as e:
    print(f"Error configuring Gemini API: {e}")
    print("Please ensure you have a 'GEMINI' secret set up in Google Colab or your API key is configured correctly.")
    # Exit or handle gracefully if API key is not available
    exit("API Key not found. Cannot proceed with LLM integration.")


# --- 1. Simple Blockchain Simulation ---
# This class simulates a very basic blockchain to record transactions (agent actions).
# It demonstrates immutability and chaining of blocks.
class SimpleBlockchain:
    def __init__(self):
        self.chain = []
        self.pending_transactions = []
        self.create_genesis_block()

    def create_genesis_block(self):
        """Creates the first block in the chain (genesis block)."""
        self.create_block(proof=1, previous_hash='0')

    def create_block(self, proof, previous_hash):
        """
        Creates a new block and adds it to the chain.
        :param proof: The proof of work (simplified).
        :param previous_hash: Hash of the previous block.
        :return: The new block.
        """
        block = {
            'index': len(self.chain) + 1,
            'timestamp': time.time(),
            'transactions': self.pending_transactions,
            'proof': proof,
            'previous_hash': previous_hash or self.hash(self.chain[-1])
        }
        self.pending_transactions = [] # Clear pending transactions
        self.chain.append(block)
        return block

    @property
    def last_block(self):
        """Returns the last block in the chain."""
        return self.chain[-1]

    @staticmethod
    def hash(block):
        """
        Hashes a block.
        :param block: A block dictionary.
        :return: SHA-256 hash string.
        """
        # We must make sure that the Dictionary is Ordered, or we'll have inconsistent hashes
        block_string = json.dumps(block, sort_keys=True).encode()
        return hashlib.sha256(block_string).hexdigest()

    def add_transaction(self, sender, recipient, amount, description):
        """
        Adds a new transaction to the list of pending transactions.
        :param sender: Identifier of the sender.
        :param recipient: Identifier of the recipient.
        :param amount: Amount of the transaction (can be any value).
        :param description: A description of the transaction/action.
        :return: The index of the block that will hold this transaction.
        """
        transaction = {
            'sender': sender,
            'recipient': recipient,
            'amount': amount,
            'description': description,
            'timestamp': time.time()
        }
        self.pending_transactions.append(transaction)
        return self.last_block['index'] + 1

    def print_chain(self):
        """Prints the entire blockchain for demonstration."""
        print("\n--- Blockchain Ledger ---")
        for block in self.chain:
            print(f"Block Index: {block['index']}")
            print(f"Timestamp: {time.ctime(block['timestamp'])}")
            print(f"Transactions: {json.dumps(block['transactions'], indent=2)}")
            print(f"Proof: {block['proof']}")
            print(f"Previous Hash: {block['previous_hash']}")
            print(f"Current Block Hash: {self.hash(block)}")
            print("-" * 30)
        print("-------------------------\n")


# --- 2. LLM Simulator (now integrated with Gemini) ---
# This class now uses the actual Gemini API to provide recommendations.
class LLMSimulator:
    def __init__(self, model_name="gemini-2.5-flash"): # Changed model to gemini-2.5-flash
        self.model_name = model_name
        try:
            self.model = genai.GenerativeModel(self.model_name)
            print(f"LLM Simulator initialized using actual Gemini '{self.model_name}' model.")
        except Exception as e:
            print(f"Error initializing Gemini model: {e}")
            self.model = None # Set to None if initialization fails

    def generate_recommendation(self, prompt: str) -> str:
        """
        Uses the Gemini model to generate a recommendation based on a prompt.
        :param prompt: The input query or context for the LLM.
        :return: A recommendation string from Gemini, or an error message.
        """
        if not self.model:
            return "RECOMMENDATION: LLM not available. REASON: Gemini model failed to initialize."

        full_prompt = (
            "You are an AI assistant for an inventory management agent. "
            "Your goal is to provide clear, actionable recommendations for 'product X'. "
            "Based on the following inventory status, provide a concise recommendation and a reason. "
            "If inventory is very low (e.g., negative or single digits), recommend 'Order X units'. "
            "If inventory is very high (e.g., over 150 units), recommend 'Initiate promotional sale for Y units'. "
            "Otherwise, recommend 'Monitor inventory levels'. "
            "Always start your response with 'RECOMMENDATION:' followed by the action, and then 'REASON:' followed by the explanation. "
            "Example 1 (low): 'RECOMMENDATION: Order 100 units of product X. REASON: Inventory is critically low and needs immediate replenishment.' "
            "Example 2 (high): 'RECOMMENDATION: Initiate promotional sale for 50 units of product X. REASON: Excess inventory detected, clear stock to avoid holding costs.' "
            "Example 3 (normal): 'RECOMMENDATION: Monitor inventory levels. REASON: Current stock is within optimal range, no immediate action required.'\n\n"
            f"Inventory Status: {prompt}"
        )
        print(f"LLM processing prompt (sending to Gemini): '{full_prompt}'...")
        try:
            response = self.model.generate_content(full_prompt)
            # Accessing the text from the response object
            if response.candidates:
                return response.candidates[0].content.parts[0].text
            else:
                return f"RECOMMENDATION: No clear response from LLM. REASON: {response.prompt_feedback}"
        except Exception as e:
            return f"RECOMMENDATION: Error during LLM call. REASON: {e}"


# --- 3. AI Agent ---
# This class represents an AI agent that interacts with the LLM and the blockchain.
# It perceives, reasons (via LLM), acts, and records.
class AIAgent:
    def __init__(self, name: str, initial_inventory: int, llm: LLMSimulator, blockchain: SimpleBlockchain):
        self.name = name
        self.inventory = initial_inventory
        self.llm = llm
        self.blockchain = blockchain
        print(f"AI Agent '{self.name}' initialized with inventory: {self.inventory}")

    def perceive_and_act(self):
        """
        The agent's main loop: perceive, get LLM recommendation, act, and record.
        """
        print(f"\n--- Agent '{self.name}' Cycle ---")
        current_inventory_status = f"Current inventory for product X is {self.inventory} units."
        print(f"Agent perceives: {current_inventory_status}")

        # Agent consults the LLM for a recommendation
        llm_response = self.llm.generate_recommendation(current_inventory_status)
        print(f"LLM Response: {llm_response}")

        action_taken = "No action"
        transaction_description = f"Agent {self.name} monitored inventory."
        transaction_amount = 0

        # Agent acts based on LLM's recommendation (simplified logic based on keywords from LLM response)
        llm_response_lower = llm_response.lower()

        # Try to extract a quantity if "order" or "buy" is mentioned
        if "order" in llm_response_lower or "buy" in llm_response_lower or "replenish" in llm_response_lower:
            match = re.search(r'(\d+)\s*units', llm_response_lower)
            order_qty = int(match.group(1)) if match else 100 # Default to 100 if no specific number

            # Agent's internal threshold for ordering (can be more sophisticated)
            if self.inventory < 50 or "immediately" in llm_response_lower or "urgently" in llm_response_lower:
                self.inventory += order_qty
                action_taken = f"Ordered {order_qty} units of product X based on LLM recommendation."
                transaction_description = f"Agent {self.name} ordered {order_qty} units of product X."
                transaction_amount = order_qty
            else:
                action_taken = "Considered ordering but inventory not critical enough based on internal threshold."
                transaction_description = f"Agent {self.name} considered ordering but decided against it (internal threshold)."

        elif "promotional sale" in llm_response_lower or "reduce excess" in llm_response_lower or "sell" in llm_response_lower:
            match = re.search(r'(\d+)\s*units', llm_response_lower)
            sale_qty = int(match.group(1)) if match else 50 # Default to 50 if no specific number

            # Agent's internal threshold for selling
            if self.inventory > 150: # Only sell if truly in excess
                self.inventory -= sale_qty
                action_taken = f"Initiated promotional sale for {sale_qty} units of product X based on LLM recommendation."
                transaction_description = f"Agent {self.name} initiated sale for {sale_qty} units of product X."
                transaction_amount = -sale_qty # Negative for reduction
            else:
                action_taken = "Considered sale but inventory not in excess based on internal threshold."
                transaction_description = f"Agent {self.name} considered sale but decided against it (internal threshold)."

        elif "monitor inventory" in llm_response_lower or "sufficient stock" in llm_response_lower or "gather more information" in llm_response_lower:
            action_taken = "Monitored inventory levels or requested more information as recommended."
            transaction_description = f"Agent {self.name} monitored inventory/requested info as recommended."
            transaction_amount = 0
        else:
            # Fallback if LLM response doesn't match any specific action keywords
            action_taken = "LLM recommendation unclear or no specific action triggered."
            transaction_description = f"Agent {self.name} received unclear LLM recommendation or no specific action."

        print(f"Agent Action: {action_taken}")
        print(f"New Inventory: {self.inventory}")

        # Record the action on the blockchain
        self.blockchain.add_transaction(
            sender=self.name,
            recipient="SupplyChainLedger",
            amount=transaction_amount,
            description=transaction_description
        )
        # In a real scenario, mining/proof of work would occur here to add the block
        # For this demo, we'll manually add a block after each agent cycle for simplicity
        # We use a simple proof based on the number of pending transactions
        self.blockchain.create_block(proof=len(self.blockchain.pending_transactions),
                                     previous_hash=self.blockchain.hash(self.blockchain.last_block))
        print(f"Action recorded on blockchain. Current chain length: {len(self.blockchain.chain)}")


# --- Main Demonstration ---
if __name__ == "__main__":
    print("Starting AI Agent, LLM (Gemini 2.5 Flash), and Blockchain Synergy Demonstration...\n") # Updated title

    # 1. Initialize the Blockchain
    supply_chain_blockchain = SimpleBlockchain()

    # 2. Initialize the LLM Simulator (now using Gemini)
    llm_brain = LLMSimulator()

    # 3. Initialize the AI Agent
    # Starting with a low inventory to encourage an "order" recommendation early
    inventory_agent = AIAgent(
        name="InventoryManagerBot",
        initial_inventory=30,
        llm=llm_brain,
        blockchain=supply_chain_blockchain
    )

    # Run several cycles of the agent's operation
    print("\n--- Running Agent Cycles ---")
    for i in range(1, 6):
        print(f"\n--- Simulation Cycle {i} ---")
        inventory_agent.perceive_and_act()
        # Simulate some time passing or external changes affecting inventory
        if i == 2:
            print("\n*Simulating a sudden demand spike: Inventory drops significantly!*")
            inventory_agent.inventory -= 50 # Simulate sales, making it negative
        elif i == 4:
            print("\n*Simulating a large incoming shipment: Inventory increases significantly!*")
            inventory_agent.inventory += 150 # Simulate new stock arrival
        time.sleep(3) # Pause for readability and to allow LLM calls to complete

    # Print the final state of the blockchain
    supply_chain_blockchain.print_chain()

    print("\nDemonstration Complete.")
    print("This conceptual demo illustrates how an AI agent, guided by Gemini 2.5 Flash, can interact with a blockchain") # Updated conclusion
    print("to manage tasks (like inventory) and maintain an immutable, transparent record of its actions.")


Gemini API configured successfully.
Starting AI Agent, LLM (Gemini 2.5 Flash), and Blockchain Synergy Demonstration...

LLM Simulator initialized using actual Gemini 'gemini-2.5-flash' model.
AI Agent 'InventoryManagerBot' initialized with inventory: 30

--- Running Agent Cycles ---

--- Simulation Cycle 1 ---

--- Agent 'InventoryManagerBot' Cycle ---
Agent perceives: Current inventory for product X is 30 units.
LLM processing prompt (sending to Gemini): 'You are an AI assistant for an inventory management agent. Your goal is to provide clear, actionable recommendations for 'product X'. Based on the following inventory status, provide a concise recommendation and a reason. If inventory is very low (e.g., negative or single digits), recommend 'Order X units'. If inventory is very high (e.g., over 150 units), recommend 'Initiate promotional sale for Y units'. Otherwise, recommend 'Monitor inventory levels'. Always start your response with 'RECOMMENDATION:' followed by the action, and th